In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1.2.- Implementar AlexNet en tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Cargar CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalizar imágenes
x_train, x_test = (x_train / 255.0) * 2 - 1, (x_test / 255.0) * 2 - 1
y_train, y_test = y_train.flatten(), y_test.flatten()

# ✅ Crear función de preprocesamiento para redimensionar dinámicamente
def preprocess(image, label):
    image = tf.image.resize(image, (227, 227))  # Se hace en cada batch, no todo a la vez
    return image, label

# ✅ Crear datasets con `tf.data` (evita OOM)
batch_size = 64

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Definir AlexNet
def build_alexnet():
    model = Sequential([
        Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=(227, 227, 3)),
        MaxPooling2D(pool_size=3, strides=2),

        Conv2D(256, kernel_size=5, padding='same', activation='relu'),
        MaxPooling2D(pool_size=3, strides=2),

        Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        Conv2D(384, kernel_size=3, padding='same', activation='relu'),
        Conv2D(256, kernel_size=3, padding='same', activation='relu'),
        MaxPooling2D(pool_size=3, strides=2),

        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model



# Crear y compilar el modelo
model = build_alexnet()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Entrenar modelo con batches pequeños
model.fit(train_ds, epochs=10, validation_data=test_ds)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 94ms/step - accuracy: 0.3504 - loss: 1.7495 - val_accuracy: 0.6438 - val_loss: 1.0133
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 130s 91ms/step - accuracy: 0.6596 - loss: 0.9717 - val_accuracy: 0.7348 - val_loss: 0.7634
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 93ms/step - accuracy: 0.7554 - loss: 0.7043 - val_accuracy: 0.7495 - val_loss: 0.7764
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 93ms/step - accuracy: 0.8130 - loss: 0.5417 - val_accuracy: 0.7784 - val_loss: 0.6636
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 77s 86ms/step - accuracy: 0.8601 - loss: 0.4066 - val_accuracy: 0.7821 - val_loss: 0.7090
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 93ms/step - accuracy: 0.8965 - loss: 0.3039 - val_accuracy: 0.7801 - val_loss: 0.7702
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 78s 87ms/step - accuracy: 0.9208 - loss: 0.2305 - val_accuracy: 0.7762 - val_loss: 0.8211
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 86ms/step - accuracy: 0.9421 - loss: 0.1686 -


#2.- Ahora vamos a comparar con los modelos preentrenados

1.   Pytorch
2.   Tensor-flow



## 2.2.- Usar modelos preentrenados en TensorFlow


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import mixed_precision

import numpy as np
import matplotlib.pyplot as plt

# Habilitar Mixed Precision para reducir uso de RAM
mixed_precision.set_global_policy('mixed_float16')

# Cargar CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalizar pixel values a [-1, 1]
x_train, x_test = (x_train / 255.0) * 2 - 1, (x_test / 255.0) * 2 - 1

# Aplanar etiquetas
y_train, y_test = y_train.flatten(), y_test.flatten()

# Función de preprocesamiento para Dataset
def preprocess(img, label):
    img = tf.image.resize(img, (224, 224))  # Redimensiona sobre la marcha
    return img, label

# Crear datasets de entrenamiento y prueba con tf.data
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(len(x_train)).map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Definir input
input_tensor = Input(shape=(224, 224, 3))

# Cargar ResNet50 pre-entrenado
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Descongelar solo las últimas 15 capas para reducir uso de memoria
for layer in base_model.layers[:-15]:
    layer.trainable = False
for layer in base_model.layers[-15:]:
    layer.trainable = True

# Añadir capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', dtype='float32')(x)  # Forzar capa en float32
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', dtype='float32')(x)
x = Dropout(0.3)(x)
output_layer = Dense(10, activation='softmax', dtype='float32')(x)

# Crear modelo
model = Model(inputs=base_model.input, outputs=output_layer)
#model = Model(inputs=[base_model.input], outputs=output_layer)

# Compilar modelo con un learning rate más bajo
model.compile(optimizer=Adam(learning_rate=0.00005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callback para reducir learning rate si la precisión se estanca
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                              factor=0.2,
                              patience=3,
                              min_lr=0.00001)

# Entrenar modelo
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=[reduce_lr]
)

# Evaluar modelo
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_187']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 91s 46ms/step - accuracy: 0.3386 - loss: 1.8026 - val_accuracy: 0.4393 - val_loss: 1.6150 - learning_rate: 5.0000e-05
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 61s 38ms/step - accuracy: 0.5310 - loss: 1.3145 - val_accuracy: 0.5246 - val_loss: 1.3532 - learning_rate: 5.0000e-05
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - accuracy: 0.5725 - loss: 1.2043 - val_accuracy: 0.3927 - val_loss: 1.9133 - learning_rate: 5.0000e-05
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 38ms/step - accuracy: 0.6013 - loss: 1.1357 - val_accuracy: 0.5654 - val_loss: 1.2304 - learning_rate: 5.0000e-05
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 61s 39ms/step - accuracy: 0.6216 - loss: 1.0712 - val_accuracy: 0.5830 - val_loss: 1.1892 - learning_rate: 5.0000e-05
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 39ms/step - accuracy: 0.6390 - loss: 1.0219 - val_accuracy: 0.5377 - val_loss: 1.3490 - learning_rate: 5.0000e-05
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 38ms/